In [1]:
import time
import os
import pandas as pd
import numpy as np
import openai
import time
openai.api_key = os.getenv("OPENAI_API_KEY")

In [2]:
data_path="../../data/"
output_path="../../outputs/"
test_set_complete=pd.read_csv(output_path+"general/test_set_final.csv")
#train_set=pd.read_csv(output_path+"general/train_set_final.csv")
#dev_set=pd.read_csv(output_path+"general/dev_set_final.csv")
df=pd.concat([test_set_complete.iloc[:2000]]).reset_index(drop=True)

In [3]:
df=df.assign(abstract_for_prompt=np.where(df.abstract_clean!="",
                                                   "Abstract: "+df.abstract_clean,""))
df=df.assign(acknowledgments_for_prompt=np.where(df.acknowledgments_clean!="",
                                                   "\nAcknowledgments: "+df.acknowledgments_clean,""))
df=df.assign(abstract_for_prompt=df.abstract_for_prompt.fillna(""))
df=df.assign(paper_text="Title: "+df.title_clean+"\n"+df.abstract_for_prompt)

ack_preprompt="""Identify the NLP task that this paper is dealing with. Select a text span that is an appropriate answer, or if no span serves as a good answer, just come up with a phrase."""

ack_postprompt="""The NLP task addressed in this paper is:"""

In [4]:
ack_preprompt="""Identify the NLP task(s) that this paper is dealing with. Select a text span that is an appropriate answer, or if no span serves as a good answer, just come up with a phrase. Examples of tasks are: fake news detection, name entity recognition, question answering, etc."""

ack_postprompt="""The primary NLP task addressed in this paper is:"""

In [5]:
df=df.assign(task_prompt_text=ack_preprompt+"\n"+df.paper_text+"\n"+ack_postprompt)

In [6]:
df=df.loc[df.label==1]

## cost estimation

## requests

gold=pd.read_json("annotation_20230114.json")

gold=pd.DataFrame(gold['data'].apply(pd.Series))

df=df.loc[df.ID.isin(gold.ID.unique())].reset_index(drop=True)

In [7]:
df.shape

(251, 21)

In [ ]:
for i,d in df.iterrows():
    input_prompt=d['task_prompt_text']
    print(input_prompt)
    print("Gold tasks: ",d['task_annotation'])
    print("Gold methods: ",d['method_annotation'])
    print("###################################")

In [ ]:
for i,d in df.iterrows():
    input_prompt=d['task_prompt_text']
    completion = openai.Completion.create(engine="text-davinci-002", prompt=input_prompt,temperature=0,max_tokens=60,logprobs=1)
    df.loc[i,'GPT3_response']=completion.choices[0].text

    print(input_prompt)
    print(completion.choices[0].text)
    print("Gold tasks: ",d['task_annotation'])
    print("Gold methods: ",d['method_annotation'])
    print("################################")
    time.sleep(3)

In [10]:
df=df.assign(clean_response=df.GPT3_response.replace("\n","",regex=True))
df['clean_response']=df['clean_response'].str.replace(r'\.$', '')

/tmp/ipykernel_1911340/1942881313.py:2: FutureWarning: The default value of regex will change from True to False in a future version.
  df['clean_response']=df['clean_response'].str.replace(r'\.$', '')


In [11]:
df.to_csv(output_path+"sg_ie/gpt3/GPT3_responses_task_davinci2_coarse2.csv",index=False)